In [1]:
import ast
import functools

import pandas as pd

from plot_all import get_key2df


pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

In [2]:
RUNNAME = '20250128_094430'
DIR = f'data/{RUNNAME}'
FILENAME_IN = f'{DIR}/df_all.csv'
FILENAME_OUT = f'{DIR}/df_all_clean.csv'


def read_df_all(filename):
    df_all = pd.read_csv(filename, low_memory=False)
    
    key2df = get_key2df(RUNNAME)
    
    df_aut_low = key2df[False, True] 
    df_aut_high = key2df[True, True] 
    df_aut = pd.concat([df_aut_low, df_aut_high])
    df_aut = df_aut.drop(columns=['Scenario ID'])
    
    df_conf_low = key2df[False, 'conf'] 
    df_conf_high = key2df[True, 'conf'] 
    df_conf = pd.concat([df_conf_low, df_conf_high])
    
    scenarios_df_all = set(df_all['Scenario ID'])
    scenarios_df_aut = set(df_aut.index.get_level_values('Scenario ID'))
    assert scenarios_df_all == scenarios_df_aut
    
    suffix_extra = ':::extra'
    assert not any(c.endswith(suffix_extra) for c in df_all.columns)
    
    assert not any(c.endswith(suffix_extra) for c in df_aut.columns)
    df_merged = df_all.merge(df_aut, how='outer', validate='many_to_one',
                             left_on='Scenario ID', right_on='Scenario ID',
                             suffixes=('', suffix_extra))
    df_merged = df_merged.drop(columns=[c for c in df_merged.columns if c.endswith(suffix_extra)])
    
    assert not any(c.endswith(suffix_extra) for c in df_aut.columns)
    df_merged = df_merged.merge(df_conf, how='outer', validate='many_to_one',
                                left_on='configuration', right_on='configuration',
                                suffixes=('', suffix_extra))
    df_merged = df_merged.drop(columns=[c for c in df_merged.columns if c.endswith(suffix_extra)])
    
    print(sorted(df_merged.columns, key=lambda s: s.lower()))
    return df_merged
    
    
df_all = read_df_all(FILENAME_IN)
df_all

['are_bridges', 'Average speed (m/s)', 'basename_scenario', 'blocked', 'can pass first', 'cautious mode', 'change of priorities', 'Collision events', 'collisions', 'Collisions rate', 'configuration', 'CP (index)', 'CSD score (AVs)', 'Current datetime', 'Cycle distance (m)', 'Date', 'dir_map', 'distance ToCP, m', 'filename', 'filename_screenshot', 'forcing', 'future missions', 'Human V0', 'i_map', 'improper parking', 'Is blocked', 'is_blocked', 'isCanPassFirstActive', 'isCanPassFirstAut', 'isCanPassFirstHum', 'isRacingThroughCrossroadAllowed', 'Linearization A', 'Linearization A (non-normalized)', 'Linearization B', 'Linearization B (non-normalized)', 'Linearization C', 'Linearization C (non-normalized)', 'Linearization D (non-normalized)', 'Linearization D0', 'Linearization D0 (non-normalized)', 'Linearization D1', 'Linearization D1 (non-normalized)', 'Linearization D2', 'Linearization D2 (non-normalized)', 'Linearization D3', 'Linearization D3 (non-normalized)', 'Maximum acceleration 

,scenario_id,vehicle_id,filename,dir_map,basename_scenario,i_map,position,passhum,slowness,forcing,filename_screenshot,are_bridges,configuration,isCanPassFirstHum,isCanPassFirstAut,reroutingsAtParked,reroutingsAtSlow,Date,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,reroutings at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized),Linearization D1 (non-normalized),Linearization D2 (non-normalized),Linearization D3 (non-normalized),Linearization D0 (non-normalized),Linearization D (non-normalized),seconds_blocked,is_blocked,Scenario ID,Vehicle ID,Collisions rate,Near-misses rate,Is blocked,CSD score (AVs)
0,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities",0,map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,False,baseline,change of priorities,../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-1.png,True,"map 1, with bridges, pos.var. 1",False,False,NaN,NaN,20250128_091627,HumanDrivenVehicle,778.12,11,9095.7,11,21,21,88,0,3,6.3,0.8,1799.9,0.3,5.6,5.1,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities",2025-01-28 09:18:47,0:02:19,0:29:59 (x12.9),"hum=false, aut=false",True,0.0,21 forcing events,"0 minor, 3 major (0:01:58: V0[604]-V2[522], 0:02:05: V0[643]-V1[90], 0:02:59: V0[85]-V1[194])","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",5.6,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,0,3,9093.5,11,NaN,"(573.7, 91.5)",547.8,546.0,777,NaN,745.3,NaN,DRIVING,"2: [778, 777]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...",NaN,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [3]:
RESOLUTION = 1  # m/step (m/pixel)


def same_value(series):
    is_ok = series.nunique(dropna=False) == 1
    if not is_ok:
        print(series)
        raise ValueError('not same values')
    return series.iloc[0]


def id2name(vehicle_id):
    return f"{"MV" if vehicle_id == 0 else "AV"} (V{vehicle_id})"


@functools.cache
def literal_eval(value):
    return ast.literal_eval(value)


def linearization_to_tuple(series):
    tup, = list(series)
    return literal_eval(tup)


def calculate_cs_score(series):
    """Converts string tuples to actual tuples and calculates the mean."""
    total_sum = 0
    count = 0
    for value in series:
        if isinstance(value, str):
            # Convert string to a tuple
            value = literal_eval(value)
        if isinstance(value, tuple):
            total_sum += sum(value)
            count += len(value)
        elif isinstance(value, (int, float)):
            total_sum += value
            count += 1
    return total_sum / count if count > 0 else None


def calculate_mean_cs_score(group, vehicle_ids):
    """Calculates the mean CS score across specified vehicle IDs."""
    cs_scores = [
        calculate_cs_score(group[group['Vehicle ID'] == vehicle_id]['Linearization C (non-normalized)'])
        for vehicle_id in vehicle_ids
    ]
    cs_scores = [score for score in cs_scores if score is not None]
    return sum(cs_scores) / len(cs_scores) if cs_scores else None


def process_df_all(df_all):
    df_in = df_all[~df_all['passhum']]
    
    # Dictionary to map Map IDs to the number of OPs
    map_to_ops = {
        1: 2, 6: 2, 10: 2,  # Maps with 2 OPs
        2: 1, 3: 1, 4: 1, 5: 1, 7: 1, 8: 1, 9: 1,  # Maps with 1 OP
    }
    
    rows = []
    for name, group in df_in.groupby('Scenario ID'):
#        group_avs = group[group['Vehicle ID'] != 0]
        
        map_id = same_value(group['i_map'])

        is_forcing = same_value(group['forcing']) != 'baseline'
        is_slowness = same_value(group['slowness']) != 'baseline'
        
        is_with_bridges = same_value(group['dir_map']).endswith('_with_bridges')
        is_without_bridges = same_value(group['dir_map']).endswith('_without_bridges')
        assert is_with_bridges != is_without_bridges

        mission_lengths = pd.Series({
            vehicle_id: 
                len(linearization_to_tuple(
                    group[group['Vehicle ID'] == vehicle_id]['Linearization C (non-normalized)']
                )) * RESOLUTION
            for vehicle_id in range(4)
        })

        ids_mv = [0]
        ids_av = [1, 2, 3]
        
        row = {
            '': {
                'Scenario ID': name,
            },
            'Violation type': {
                'Priority violation': is_forcing and not is_slowness,
                'Speed violation': is_slowness and not is_forcing,
                'Both': is_forcing and is_slowness,
            },
            'Coordination strategy': {
                'Change of priorities': same_value(group['forcing']) == 'change of priorities',
                'Stops': same_value(group['forcing']) == 'stops',
                'Rerouting': same_value(group['slowness']) == 'with rerouting',
            },
            'Static map features': {
                '~ Map ID': map_id,
                'No. of OPs': map_to_ops.get(map_id, None),
                'Connectivity': 'high' if same_value(group['dir_map']).endswith('_with_bridges') else 'low',
                '~ Position': f"{map_id}-{same_value(group['position'])}",
            },
            'Output of simulation (planning)': {
                
                # CSD (Critical Section Density) score for MV (V0)
                'CSD score for MV (V0)': calculate_cs_score(group[group['Vehicle ID'] == 0]['Linearization C (non-normalized)']),

                # CSD score for each AV
                **{
                    f'CSD score for AV (V{vehicle_id})': calculate_cs_score(group[group['Vehicle ID'] == vehicle_id]['Linearization C (non-normalized)'])
                    for vehicle_id in range(1, 4)
                },
                # Mean CSD score for AVs (V1-V3)
                'Mean CSD score for AVs (V1-V3)': same_value(group['CSD score (AVs)']),
                
                # Mission length for each AV (V1-V3)
                **{
                    f'Mission length for {id2name(vehicle_id)}': mission_lengths[vehicle_id]
                    for vehicle_id in range(4)
                },
                # Total and mean mission lengths for AVs (V1-V3)
                'Total Mission length for AVs (V1-V3)': mission_lengths[ids_av].sum(),
                'Mean Mission length for AVs (V1-V3)': mission_lengths[ids_av].mean(),
            },
            'Output of simulation (execution)': functools.reduce(lambda x, y: x | y, (
                {
                    ('~ ' if vehicle_id == 0 else '') + f'{column} for {id2name(vehicle_id)}': 
                        group[group['Vehicle ID'] == vehicle_id][column].sum()
                    for vehicle_id in range(4)
                } | {
                    f'Total {column} for AVs (V1-V3)': (
                        group[column].sum() // 2
                        if column in ('No. of collisions', 'No. of near-misses')
                        else
                        group[group['Vehicle ID'].isin(ids_av)][column].sum()
                    ),
                } | (
                    {
                        'Collisions rate': same_value(group['Collisions rate']), 
                        # 'No. of violations': group['No. of violations'].sum(),  # to check
                    }
                    if column == 'No. of collisions' else
                    {'Near-misses rate': same_value(group['Near-misses rate'])}
                    if column == 'No. of near-misses' else
                    {}
                )
                for column in (
                    'No. of completed missions',
                    'No. of collisions',
                    'No. of near-misses',
                )
            )),
        }
        rows.append(row)
        
    rows_flat = []
    for row in rows:
        row_flat = {}
        for group, values in row.items():
            for col, value in values.items():
                row_flat[group, col] = value
        rows_flat.append(row_flat)
        
    df_out = pd.DataFrame(rows_flat).round(3)
    df_out = df_out.reindex(columns=pd.MultiIndex.from_tuples(list(rows_flat[0])))
    
    df_out.to_csv(FILENAME_OUT, index=False)
    return df_out


df_processed = process_df_all(df_all)
df_processed

\
                                                                                                                                                 Scenario ID   
0                       map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities   
1                             map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human   
2                                         map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no   
3                                      map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops   
4           map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing change of priorities   
...                                                                                                                                                      ...   
1895                                map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing stops   
1896  map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities   
1897        map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing ignoring human   
1898                    map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing no   
1899                 map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops   

         Violation type                        Coordination strategy         \
     Priority violation Speed violation   Both  Change of priorities  Stops   
0                  True           False  False                  True  False   
1                  True           False  False                 False  False   
2                 False           False  False                 False  False   
3                  True           False  False                 False   True   
4                 False           False   True                  True  False   
...                 ...             ...    ...                   ...    ...   
1895               True           False  False                 False   True   
1896              False           False   True                  True  False   
1897              False           False   True                 False  False   
1898              False            True  False                 False  False   
1899              False           False   True                 False   True   

               Static map features                                     \
     Rerouting            ~ Map ID No. of OPs Connectivity ~ Position   
0        False                   1          2         high        1-1   
1        False                   1          2         high        1-1   
2        False                   1          2         high        1-1   
3        False                   1          2         high        1-1   
4         True                   1          2         high        1-1   
...        ...                 ...        ...          ...        ...   
1895     False                   9          1          low        9-9   
1896     False                   9          1          low        9-9   
1897     False                   9          1          low        9-9   
1898     False                   9          1          low        9-9   
1899     False                   9          1          low        9-9   

     Output of simulation (planning)                        \
               CSD score for MV (V0) 

In [4]:
pd.read_csv(FILENAME_OUT, header=list(range(2)), index_col=0)

Violation type  \
Scenario ID                                                                                                                                            Priority violation   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                                 True   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                                       True   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no                                                  False   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops                                                True   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing change of priorities                    False   
...                                                                                                                                                                   ...   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing stops                                             True   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities              False   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing ignoring human                    False   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing no                                False   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops                             False   

                                                                                                                                                                        \
Scenario ID                                                                                                                                            Speed violation   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities                             False   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing ignoring human                                   False   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no                                               False   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops                                            False   
map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing change of priorities                 False   
...                                                                                                                                                                ...   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing stops                                         False   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities           False   
map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing ignoring human                 False   
map-generator/generated-maps/2024-1